# Augmentation Library Imgaug 설치

In [26]:
# !pip install imgaug

# 라이브러리 불러오기

In [27]:
import glob
import imageio
import imgaug as ia
from imgaug import augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
import xml.etree.ElementTree as ET
from PIL import Image
import os
%matplotlib inline

# 변수 세팅

In [28]:
label_info_path = 'classes.txt' # 텍스트 파일 만들고 label 을 순서대로 적어 놓자 
# aug folder path
aug_folder = 'data'
# # img_file_path
# img_file_path = aug_folder + "/*.jpg"
# img_file_path2 = aug_folder + "/*.png"
#xml_file_path 
xml_file_path = aug_folder + "/*.xml"
#txt_file_path
txt_file_path = aug_folder + "/*.txt"

#save_dir
save_dir = "aug"
#save_label
save_label = save_dir +"\\"+ "label"
#save_img
save_img = save_dir +"\\"+"img"

#resize box size
box_size = 416
# Augmentation을 할 옵션들을 선택
augs = ["Add","Dropout","MotionBlur","AddtiveGaussianNoise","Sharpen","Emboss","Scale","Shear","CropAndPad","WithBrightnessChannels","HorizontalFlip"]
#['Identity']

#input format - xml, txt
input_format = 'xml'
#save format, yolo, xml 지금은 yolo만 구현
save_format = 'yolo'

#반복 횟수
iter_num = 10

## label mapping 중

In [29]:
labelmap ={}
labellist = []
with open(label_info_path, mode='r', encoding='utf-8') as f:
    label_list = f.readlines()
for i,label in enumerate(label_list):
    labelmap[label.replace('\n',"")] = i
    labellist.append(label.replace('\n',""))

# augmentation 할 파일 경로 리스트 세팅

In [30]:
## augmentation 할 파일 경로 리스트 세팅

# img_path_list = glob.glob(img_file_path)
# img_path_list.append(glob.glob(img_file_path2))
if input_format == 'xml':
    xml_path_list = glob.glob(xml_file_path)
elif input_format == 'txt':        
    txt_path_list = glob.glob(txt_file_path)

In [31]:
# xml_path_list

In [32]:
# txt_path_list

# Augmentation 기능 매핑 및 추가

링크 타고 가서 어떤 효과인지는 찾아보세요 
https://imgaug.readthedocs.io/en/latest/source/overview/geometric.html


In [33]:
# aug 종류
#너무 많아서 손아파서 못쓰겠다 알아서 써라 


# augmentation 리스트 초기화
aug_list = []

if "Identity" in augs: #아무것도 안함
    aug_list.append(iaa.Identity())
if "Add" in augs:
    aug_list.append(iaa.Add((-40,40)))
if "AddElementwise" in augs:
    aug_list.append(iaa.AddElementwise((-40, 40)))
if "AddtiveGaussianNoise" in augs:
    aug_list.append(iaa.AdditiveGaussianNoise(scale=(0, 0.2*255), per_channel=True))
    aug_list.append(iaa.AdditiveGaussianNoise(scale=(0, 0.2*255)))
if "AddtiveLaplaceNoise" in augs:
    aug_list.append(iaa.AddtiveLaplaceNoise(scale=(0,0.2*255)))
    aug_list.append(iaa.AddtiveLaplaceNoise(scale=(0,0.2*255), per_channel=True))
if "AddtivePoissonNoise" in augs:
    aug_list.append(iaa.AdditivePoissonNoise(scale=(0, 40)))
if "Multiply" in augs:
    aug_list.append(iaa.Multiply((0.5, 1.5)))
if "MultiplyElementwise" in augs:
    aug_list.append(iaa.Cutout(nb_iterations=(1,5), fill_mode="gaussian", fill_per_channel=True))
    aug_list.append(iaa.Cutout(fill_mode="gaussian", fill_per_channel=False))
if "Dropout" in augs:
    aug_list.append(iaa.Dropout(p=(0,0.2)))
    aug_list.append(iaa.Dropout(p=(0,0.2), per_channel=0.5))
if "CoarseDropout" in augs:
    aug_list.append(iaa.CoarseDropout((0.0, 0.05), size_percent=(0.02,0.25)))
    aug_list.append(iaa.CoarseDropout(0.02, size_percent=0.15, per_channel=0.5))
if "ImpulseNoise" in augs:
    aug_list.append(iaa.ImpulseNoise(0.1))
if "SaltAndPepper" in augs:
    aug_list.append(iaa.SaltAndPepper(0.1), per_channel=True)
if "CoarseSaltAndPepper" in augs:
    aug_list.append(iaa.CoarseSaltAndPepper(
    0.05, size_percent=(0.01, 0.1), per_channel=True))
if "Invert" in augs:
    aug_list.append(iaa.Invert(0.25, per_channel=0.5))
if "Solarize" in augs:
    aug_list.append(iaa.Solarize(0.5, threshold=(32,128)))
if "JpegComppression" in augs:
    aug_list.append(iaa.JpegCompression(compression=(70,99)))
if "Cartoon" in augs:
    aug_list.append(iaa.Cartoon())
if "GaussianBlur" in augs:
    aug_list.append(iaa.GaussianBlur(sigma=(0.0,3.0)))
if "AverageBlur" in augs:
    aug_list.append(iaa.AverageBlur(k=(2,11)))
if "MedianBlur" in augs:
    aug_list.append(iaa.MedianBlur(k=(3,11)))
if "BilateralBlur" in augs:
    aug_list.append(iaa.BilateralBlur(
    d=(3, 10), sigma_color=(10, 250), sigma_space=(10, 250)))
if "MotionBlur" in augs:
    aug_list.append(iaa.MotionBlur(k=10,angle=[-45, 45]))
if "MeanShiftBlur" in augs:
    aug_list.append(iaa.MeanShiftBlur())
if "RandAugment" in augs: # 아무거나 적용하는 듯?
    aug_list.append(iaa.RandAugment(n=2,m=9))
    aug_list.append(iaa.RandAugment(m=30))
    aug_list.append(iaa.RandAugment(m=(0,9)))
if "WithColorspace" in augs:
    aug_list.append(iaa.WithColorspace(
    to_colorspace="HSV",
    from_colorspace="RGB",
    children=iaa.WithChannels(
        0,
        iaa.Add((0, 50))
    )
))
if "WithBrightnessChannels" in augs:
    aug_list.append(iaa.WithBrightnessChannels(iaa.Add((-50, 50))))
if "ChangeColorTemperature" in augs:
    aug_list.append(iaa.ChangeColorTemperature((1100, 10000)))
if "KMeansColorQuantization" in augs:
    aug_list.append(iaa.KMeansColorQuantization(n_colors=(4, 16)))
if "UniformColorQuantization" in augs:
    aug_list.append(iaa.UniformColorQuantization(n_colors=(4, 16)))
if "HorizontalFlip" in augs:
    aug_list.append(iaa.HorizontalFlip())
if "VerticalFlip" in augs:
    aug_list.append(iaa.VerticalFlip())
if "Fliplr" in augs:
    aug_list.append(iaa.Fliplr(0.5))
if "Fog" in augs:
    aug_list.append(iaa.imgcorruplike.Fog(severity=2))
if "Frost" in augs:
    aug_list.append(iaa.imgcorruplike.Frost(severity=2))
if "Snow" in augs:
    aug_list.append(iaa.imgcorruplike.Snow(severity=2))
if "Spatter" in augs:
    aug_list.append(iaa.imgcorruplike.Spatter(severity=2))
if "Pixelate" in augs:
    aug_list.append(iaa.imgcorruptlike.Pixelate(severity=2))
if "FastSnowyLandscape" in augs:
    aug_list.append(iaa.FastSnowyLandscape(
    lightness_threshold=[128, 200],
    lightness_multiplier=(1.5, 3.5)
))
if "Clouds" in augs:
    aug_list.append(iaa.Clouds())
if "Snowflakes" in augs:
    aug_list.append(flake_size=(0.2, 0.7), speed=(0.007, 0.03))
if "Rain" in augs:
    aug_list.append(iaa.Rain(speed=(0.1, 0.3)))
if "GammaContrast" in augs:
    aug_list.append(iaa.GammaContrast((0.5, 2.0)))
if "SigmoidContrast" in augs:
    aug_list.append(iaa.SigmoidContrast(gain=(3, 10), cutoff=(0.4, 0.6)))
if "LogContrast" in augs:
    aug_list.append(iaa.LogContrast(gain=(0.6,1.4)))
if "LinearContrast" in augs:
    aug_list.append(iaa.LinearContrast((0.4,1.6)))
if "CKAGE" in augs:
    aug_list.append(iaa.AllChannelsCLAHE(clip_limit=(1, 10)))
if "AllChannelsHistogramEqualization" in augs:
    aug_list.append(iaa.Alpha((0.0, 1.0), iaa.AllChannelsHistogramEqualization()))
if "Convolve" in augs:
    matrix = np.array([[0, -1, 0],
                   [-1, 4, -1],
                   [0, -1, 0]])
    aug_list.append(iaa.Convolve(matrix=matrix))
if "Sharpen" in augs:
    aug_list.append(iaa.Sharpen(alpha=(0.0, 1.0), lightness=(0.75, 2.0)))
if "Emboss" in augs:
    aug_list.append(iaa.Emboss(alpha=(0.0,1.0), strength=(0.5,1.5)))
if "Scale" in augs:
    aug_list.append(iaa.Affine(scale=(0.5, 1.5)))
if "Rotate" in augs:
    aug_list.append(iaa.Affine(rotate=(-45,45)))
if "Shear" in augs:
    aug_list.append(iaa.Affine(shear=(-16,16)))
if "PiecewiseAffine" in augs:
    aug_list.append( iaa.PiecewiseAffine(scale=(0.01, 0.05)))
if "PerspectiveTransform" in augs:
    aug_list.append(iaa.PerspectiveTransform(scale=(0.01, 0.15)))
if "ElasticTransformation" in augs:
    aug_list.append(iaa.ElasticTransformation(alpha=(0, 5.0), sigma=0.25))
if "CropAndPad" in augs:
    aug_list.append(iaa.CropAndPad(
        percent=(-0.05, 0.1),
            pad_mode=ia.ALL,
            pad_cval=(0, 255)
    ))

In [34]:
aug_list

[Add(name=UnnamedAdd, parameters=[Uniform(Deterministic(int -40), Deterministic(int 40)), Deterministic(int 0)], deterministic=False),
 AdditiveGaussianNoise(name=UnnamedAdditiveGaussianNoise, parameters=[Normal(loc=Deterministic(int 0), scale=Uniform(Deterministic(int 0), Deterministic(float 51.00000000))), Deterministic(int 1)], deterministic=False),
 AdditiveGaussianNoise(name=UnnamedAdditiveGaussianNoise, parameters=[Normal(loc=Deterministic(int 0), scale=Uniform(Deterministic(int 0), Deterministic(float 51.00000000))), Deterministic(int 0)], deterministic=False),
 Dropout(name=UnnamedDropout, parameters=[Binomial(Uniform(Deterministic(float 0.80000000), Deterministic(int 1))), Deterministic(int 0)], deterministic=False),
 Dropout(name=UnnamedDropout, parameters=[Binomial(Uniform(Deterministic(float 0.80000000), Deterministic(int 1))), Binomial(Deterministic(float 0.50000000))], deterministic=False),
 MotionBlur(name=UnnamedMotionBlur, parameters=[<imgaug.augmenters.blur._MotionBlu

# Augmentation 객체 생성

In [35]:
seq = iaa.Sequential( ## 선택된 augmentator 중에서 0~전체 개수의 augmentator에서 랜덤한 순서대로 선택하여 진행
    iaa.SomeOf((0, None),
              aug_list,
              random_order=True)
)
seq2 = iaa.Sequential( ## 하나만 랜덤하게 골라서 진행
    iaa.OneOf(aug_list)
)

# VOC 형식의 XML 데이터에서 바운딩 박스와 라벨 데이터 추출하는 함수

In [36]:
def getXMLDATA(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot() # 루트 파싱
    file_info ={}
    objects_list = []
    x = file_path.split('\\')
    x.reverse()
    #파일 명 추가
    file_info['filename'] = x[0]
    #사진 크기 추가
    file_info['width'] = root.find("size").find("width").text
    file_info['height'] = root.find("size").find("height").text
    #object 리스트 추가 
    for o in root.iter('object'):
        bndbox = {}
        bndbox['label'] = o.find("name").text # 라벨링 추가 
        sizes = []
        sizes.append(o.find("bndbox").find("xmin").text)
        sizes.append(o.find("bndbox").find("ymin").text)
        sizes.append(o.find("bndbox").find("xmax").text)
        sizes.append(o.find("bndbox").find("ymax").text)
        bndbox["bndbox"] = sizes
        objects_list.append(bndbox)
    file_info['objects'] = objects_list
    return file_info

# yolov Darknet 의 txt 데이터에서 바운딩 박스와 라벨 데이터 추출

In [37]:
def getImgSize(img_path):
    image = Image.open(img_path)
    width, height = image.size
    size =[]
    size.append(width)
    size.append(height)
    return size

def getYoloDATA(file_path, img_size):
    with open(file_path, mode='r', encoding='utf-8') as f:
        datas=f.readlines()
    file_info = {}
    x = file_path.split('\\')
    x.reverse()
    #파일 명 추가
    file_info['filename'] = x[0]
    #사진 크기 추가
    file_info['width'] = img_size[0]
    file_info['height'] = img_size[1]
    #object 리스트 추가
    objects_list = []
    for i in datas:
        i = i.replace("\n","")
        bndbox = {}
        temp = i.split(' ')
        bndbox['label'] = temp[0]
        sizes = []
        xmax = (float(temp[1])*img_size[0]*2 + float(temp[3])*img_size[0])/2.0
        xmin = (float(temp[1])*img_size[0]*2 - float(temp[3])*img_size[0])/2.0
        ymax = (float(temp[2])*img_size[1]*2 + float(temp[4])*img_size[1])/2.0
        ymin = (float(temp[2])*img_size[1]*2 - float(temp[4])*img_size[1])/2.0
        sizes.append(xmin)
        sizes.append(ymin)
        sizes.append(xmax)
        sizes.append(ymax)
        bndbox['bndbox'] = sizes
        objects_list.append(bndbox)
    file_info['objects'] = objects_list
    return file_info
        


# XML file Augmentation 진행 ->save yolo

In [ ]:
if input_format == 'xml':
    for k in range(iter_num):
        for i, path in enumerate(xml_path_list):
            data = getXMLDATA(path)
            #확장자 버리고 이름만 따기
            name = data['filename'].split('.')
            name = name[0]
            #이미지 읽어오기
            img_path = ""
            if os.path.isfile(aug_folder+'\\'+name+'.jpg'):
                img_path +=  aug_folder+'\\'+name+'.jpg'
            else:
                img_path +=  aug_folder+'\\'+name+'.png'
            image = imageio.imread(img_path)
            #이미지 416 으로 리사이즈
            image = ia.imresize_single_image(image,(box_size,box_size))

            #resize 이후 라벨링 비율을 바꾸기 위함
            x_ratio = box_size/int(data["width"])
            y_ratio = box_size/int(data["height"])

            # 라벨 데이터와 바운딩 박스를 분리
            objects = data["objects"]

            text_list = []
            bbs_list = []

            for obj in objects:
                text_list.append(str(labelmap[obj['label']])) #라벨들을 미리 숫자로 바꾸고 text 앞부분에 저장        
                bbs = BoundingBox(x1=round(float(obj['bndbox'][0])*x_ratio),
                                  y1=round(float(obj['bndbox'][1])*y_ratio),
                                  x2=round(float(obj['bndbox'][2])*x_ratio),
                                  y2=round(float(obj['bndbox'][3])*y_ratio))
                bbs_list.append(bbs)
        #         ia.imshow(bbs.draw_on_image(image, size=2)) # 이미지 출력해서 원본 이미지 라벨링 보고 싶다면 ㄱ
            #바운딩 박스 객체 생성
            bounding_box = BoundingBoxesOnImage(bbs_list, shape=image.shape)
            # augmentation 진행
            img_aug, bbs_aug = seq(image=image, bounding_boxes=bounding_box)
            

        #     ia.imshow(bbs_aug.draw_on_image(img_aug, size=2)) # 변환된 이미지 보고 싶다면 ㄱ

            # numpy 형식의 데이터인 이미지를 저장
            try: # 폴더 경로가 없는 경우 생성
                if not(os.path.isdir(save_img)):
                    os.makedirs(os.path.join(save_img))
                if not(os.path.isdir(save_label)):
                    os.makedirs(os.path.join(save_label))
            except OSError as e:
                if e.errno != errno.EXIST:
                    print("Failed to create directory!!!!!")
                    raise


            if save_format == "yolo":
                im = Image.fromarray(img_aug)
                im.save(save_img+"\\"+name+'_aug' + str(k)+ '.jpg')


                for i,box in enumerate(bbs_aug):
                    dw = 1./box_size
                    dh = 1./box_size
                    if(box[1][0] > box_size): box[1][0] = box_size
                    if(box[1][1] > box_size): box[1][1] = box_size
                    x = ((int(box[0][0])+int(box[1][0]))/2.0)*dw
                    y = ((int(box[0][1])+int(box[1][1]))/2.0)*dh
                    w = (int(box[1][0]) - int(box[0][0]))*dw                
                    h = (int(box[1][1]) - int(box[0][1]))*dh      
                    
                    if x>1: x = 1
                    if y>1: y = 1
                    if w>1: w = 1
                    if h>1: h = 1
                        
                    text_list[i] += ' ' + str(x) +' '+ str(y)  +' ' + str(w) +' ' +str(h)

                for i in text_list:
                    with open(save_label+"\\"+name+'_aug' + str(k)+ '.txt', mode='wt', encoding='utf-8') as f:
                        f.write(i+'\n')

    with open(save_img+'\\'+"classes.labels", mode='wt', encoding='utf-8') as f:
        for i in labellist:
            f.write(i+'\n')

# Yolo augmentation ->yolo save

In [ ]:
if input_format == 'txt':
    for k in range(iter_num):
        for i, path in enumerate(txt_path_list):
            
            #확장자 버리고 이름만 따기
            temp = path.split('\\')
            temp.reverse()
            txt_file_name = temp[0].split('.txt')
            name = txt_file_name[0]
            #이미지 읽어오기
            img_path = ""
            if os.path.isfile(aug_folder+'\\'+name+'.jpg'):
                img_path +=  aug_folder+'\\'+name+'.jpg'
            else:
                img_path +=  aug_folder+'\\'+name+'.png'
            image = imageio.imread(img_path)
            
            #이미지 416 으로 리사이즈
            image = ia.imresize_single_image(image,(box_size,box_size))
            
            #txt 데이터 
            data = getYoloDATA(path,getImgSize(img_path))

            #resize 이후 라벨링 비율을 바꾸기 위함
            x_ratio = box_size/int(data["width"])
            y_ratio = box_size/int(data["height"])

            # 라벨 데이터와 바운딩 박스를 분리
            objects = data["objects"]

            text_list = []
            bbs_list = []

            for obj in objects:
                text_list.append(str(obj['label'])) #라벨들을 미리 숫자로 바꾸고 text 앞부분에 저장        
                bbs = BoundingBox(x1=round(float(obj['bndbox'][0])*x_ratio),
                                  y1=round(float(obj['bndbox'][1])*y_ratio),
                                  x2=round(float(obj['bndbox'][2])*x_ratio),
                                  y2=round(float(obj['bndbox'][3])*y_ratio))
                bbs_list.append(bbs)
        #         ia.imshow(bbs.draw_on_image(image, size=2)) # 이미지 출력해서 원본 이미지 라벨링 보고 싶다면 ㄱ
            #바운딩 박스 객체 생성
            bounding_box = BoundingBoxesOnImage(bbs_list, shape=image.shape)
            # augmentation 진행
            img_aug, bbs_aug = seq(image=image, bounding_boxes=bounding_box)


        #     ia.imshow(bbs_aug.draw_on_image(img_aug, size=2)) # 변환된 이미지 보고 싶다면 ㄱ

            # numpy 형식의 데이터인 이미지를 저장
            try: # 폴더 경로가 없는 경우 생성
                if not(os.path.isdir(save_img)):
                    os.makedirs(os.path.join(save_img))
                if not(os.path.isdir(save_label)):
                    os.makedirs(os.path.join(save_label))
            except OSError as e:
                if e.errno != errno.EXIST:
                    print("Failed to create directory!!!!!")
                    raise


            if save_format == "yolo":
                im = Image.fromarray(img_aug)
                im.save(save_img+"\\"+name+'_aug' + str(k)+ '.jpg')

                for i,box in enumerate(bbs_aug):
                    dw = 1./box_size
                    dh = 1./box_size
                    if(box[1][0] > box_size): box[1][0] = box_size
                    if(box[1][1] > box_size): box[1][1] = box_size
                    x = ((int(box[0][0])+int(box[1][0]))/2.0)*dw
                    y = ((int(box[0][1])+int(box[1][1]))/2.0)*dh
                    w = (int(box[1][0]) - int(box[0][0]))*dw
                    h = (int(box[1][1]) - int(box[0][1]))*dh
                    
                    if x>1: x = 1
                    if y>1: y = 1
                    if w>1: w = 1
                    if h>1: h = 1
                    text_list[i] += ' ' + str(x) +' '+ str(y)  +' ' + str(w) +' ' +str(h)

                for i in text_list:
                    with open(save_label+"\\"+name+'_aug' + str(k)+ '.txt', mode='wt', encoding='utf-8') as f:
                        f.write(i+'\n')

        with open(save_img+'\\'+"classes.labels", mode='wt', encoding='utf-8') as f:
            for i in labellist:
                f.write(i+'\n')

In [ ]:
# if input_format == 'xml':
#     for k in range(1):
#         for i, path in enumerate(xml_path_list):
#             data = getXMLDATA(path)
#             #확장자 버리고 이름만 따기
#             name = data['filename'].split('.')
#             name = name[0]
#             #이미지 읽어오기
#             img_path = ""
#             if os.path.isfile(aug_folder+'\\'+name+'.jpg'):
#                 img_path +=  aug_folder+'\\'+name+'.jpg'
#             else:
#                 img_path +=  aug_folder+'\\'+name+'.png'
#             image = imageio.imread(img_path)
#             #이미지 416 으로 리사이즈
#             image = ia.imresize_single_image(image,(box_size,box_size))

#             #resize 이후 라벨링 비율을 바꾸기 위함
#             x_ratio = box_size/int(data["width"])
#             y_ratio = box_size/int(data["height"])

#             # 라벨 데이터와 바운딩 박스를 분리
#             objects = data["objects"]

#             text_list = []
#             bbs_list = []

#             for obj in objects:
#                 text_list.append(str(labelmap[obj['label']])) #라벨들을 미리 숫자로 바꾸고 text 앞부분에 저장        
#                 bbs = BoundingBox(x1=round(float(obj['bndbox'][0])*x_ratio),
#                                   y1=round(float(obj['bndbox'][1])*y_ratio),
#                                   x2=round(float(obj['bndbox'][2])*x_ratio),
#                                   y2=round(float(obj['bndbox'][3])*y_ratio))
#                 bbs_list.append(bbs)
#         #         ia.imshow(bbs.draw_on_image(image, size=2)) # 이미지 출력해서 원본 이미지 라벨링 보고 싶다면 ㄱ
#             #바운딩 박스 객체 생성
#             bounding_box = BoundingBoxesOnImage(bbs_list, shape=image.shape)
#             # augmentation 진행
#             img_aug, bbs_aug = seq(image=image, bounding_boxes=bounding_box)
            

#         #     ia.imshow(bbs_aug.draw_on_image(img_aug, size=2)) # 변환된 이미지 보고 싶다면 ㄱ

#             # numpy 형식의 데이터인 이미지를 저장
#             try: # 폴더 경로가 없는 경우 생성
#                 if not(os.path.isdir(save_img)):
#                     os.makedirs(os.path.join(save_img))
#                 if not(os.path.isdir(save_label)):
#                     os.makedirs(os.path.join(save_label))
#             except OSError as e:
#                 if e.errno != errno.EXIST:
#                     print("Failed to create directory!!!!!")
#                     raise


#             if save_format == "yolo":
#                 im = Image.fromarray(img_aug)
#                 im.save(save_img+"\\"+name+'_aug' + str(k)+ '.jpg')


#                 for i,box in enumerate(bbs_aug):
#                     dw = 1./box_size
#                     dh = 1./box_size
#                     if(box[1][0] > box_size): box[1][0] = box_size
#                     if(box[1][1] > box_size): box[1][1] = box_size
#                     x = ((int(box[0][0])+int(box[1][0]))/2.0)*dw
#                     y = ((int(box[0][1])+int(box[1][1]))/2.0)*dh
#                     w = (int(box[1][0]) - int(box[0][0]))*dw                
#                     h = (int(box[1][1]) - int(box[0][1]))*dh        
#                     text_list[i] += ' ' + str(x) +' '+ str(y)  +' ' + str(w) +' ' +str(h)

#                 for i in text_list:
#                     with open(save_label+"\\"+name+'_org_resize' + str(k)+ '.txt', mode='wt', encoding='utf-8') as f:
#                         f.write(i+'\n')

#     with open(save_img+'\\'+"classes.labels", mode='wt', encoding='utf-8') as f:
#         for i in labellist:
#             f.write(i+'\n')